## Overlap of Political Organisations and Authors of Academic Research

We look at the overlap between members of various organisations and the authors of the studies in REMP, IM and IMR:

- REMP Board
- ICEM directors and deputy directors
- Dutch Government



In [1]:
import pandas as pd

records_file = '../data/main-review-article-records.csv'

# load the csv data into a data frame
pub_df = pd.read_csv(records_file)
# show the first and last records of the dataset to demonstrate what the records look like
pub_df

,Unnamed: 0,article_title,article_doi,article_author,article_author_index_name,article_author_affiliation,article_page_range,article_pub_date,article_pub_year,issue_section,issue_number,issue_title,issue_page_range,issue_pub_date,issue_pub_year,volume,journal,publisher,article_type
0,9,The importance of emigration for the solution ...,NaN,"Wander, H.","Wander, H.",NaN,NaN,1951,1951,article,NaN,NaN,NaN,1951,1951,1,Publications of the research group for europea...,Staatsdrukkerij,main
1,11,European emigration overseas past and future,NaN,"Citroen, H.A.","Citroen, H.A.",NaN,NaN,1951,1951,article,NaN,NaN,NaN,1951,1951,2,Publications of the research group for europea...,Staatsdrukkerij,main
2,13,Some aspects of migration problems in the Neth...,NaN,"Beijer, G. && Oudegeest, J.J.","Beijer, G. && Oudegeest, J.J.",&&,NaN,1952,1952,article,NaN,NaN,NaN,1952,1952,3_1,Publications of the research group for europea...,Staatsdrukkerij,main
3,15,Some quantitative aspects of future population...,NaN,"Brink, van den, T.","Brink, van den, T.",NaN,NaN,1952,1952,article,NaN,NaN,NaN,1952,1952,3_2,Publications of the research group for europea...,Staatsdrukkerij,main
4,17,"The refugees as a burden, a stimulus, and a ch...",NaN,"Edding, F.","Edding, F.",NaN,NaN,1951,1951,article,NaN,NaN,NaN,1951,1951,4,Publications of the research group for europea...,Staatsdrukkerij,main
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4279,3907,Chapter Four: Going North,https://journals.sagepub.com/doi/pdf/10.1177/0...,NaN,NaN,NaN,63-76,"Jan 1, 1986",1986,Article,1_suppl,"International Migration Review - Volume 20, Nu...",NaN,"Jan 01, 1986",1986,20,International Migration Review,Sage Publishing,main
4280,3908,Chapter Five: The New York Labor Market,https://journals.sagepub.com/doi/pdf/10.1177/0...,NaN,NaN,NaN,77-109,"Jan 1, 1986",1986,Article,1_suppl,"International Migration Review - Volume 20, Nu...",NaN,"Jan 01, 1986",1986,20,International Migration Review,Sage Publishing,main
4281,3909,Chapter Six: The Settlement Process,https://journals.sagepub.com/doi/pdf/10.1177/0...,NaN,NaN,NaN,110-119,"Jan 1, 1986",1986,Article,1_suppl,"International Migration Review - Volume 20, Nu...",NaN,"Jan 01, 1986",1986,20,International Migration Review,Sage Publishing,main
4282,3910,Chapter Seven: Conclusion,https://journals.sagepub.com/doi/pdf/10.1177/0...,NaN,NaN,NaN,120-133,"Jan 1, 1986",1986,Article,1_suppl,"International Migration Review - Volume 20, Nu...",NaN,"Jan 01, 1986",1986,20,International Migration Review,Sage Publishing,main


In [2]:
def map_dataset(publisher, article_type):
    # all REMP and IM (published by Wiley) articles are bunlded in a single dataset
    if publisher == 'Staatsdrukkerij' or publisher == 'Wiley':
        return 'REMP_IM'
    # The IMR articles are separated in review articles and research articles
    return 'IMR_research' if article_type == 'main' else 'IMR_review'

pub_df['dataset'] = pub_df.apply(lambda x: map_dataset(x['publisher'], x['article_type']), axis=1)
pub_df.dataset.value_counts()

IMR_review      1842
IMR_research    1539
REMP_IM          903
Name: dataset, dtype: int64

In [3]:
columns = ['article_title', 'issue_pub_year', 'publisher', 'article_type']


In [4]:
pub_df.article_type.value_counts()

main      2399
review    1885
Name: article_type, dtype: int64

In [5]:
pub_df.article_author
temp_df = pub_df[pub_df.article_author.isna() == False]
temp_df[temp_df.article_author.str.contains('Prof')] # Not in article_author_index_name
temp_df[temp_df.article_author.str.contains('HAVEMAN')]
temp_df[temp_df.article_author.str.contains('QUADE')]
temp_df[temp_df.article_author.str.contains('POR')] # POR B.W. HAVEMAN
temp_df[temp_df.article_author.str.contains(' De ')] # Comentario Critico De T. Cnossen

temp_df[temp_df.article_author.str.contains('Father')] # Busa, Father Roberto and Ávila, Father Fernando Bastos
temp_df[temp_df.article_author.str.contains('Comentario')] # Comentario Critico De T. Cnossen -> Cnosses, de T
temp_df[temp_df.article_author.str.contains('Avila')] # Bastos de Avila, F. and Avila, de, F.B.
temp_df[temp_df.article_author.str.contains('\u00c1vila')] # Father Fernando Bastos de Ávila S.J.
temp_df[temp_df.article_author.str.contains('Hof')] # 


,Unnamed: 0,article_title,article_doi,article_author,article_author_index_name,article_author_affiliation,article_page_range,article_pub_date,article_pub_year,issue_section,issue_number,issue_title,issue_page_range,issue_pub_date,issue_pub_year,volume,journal,publisher,article_type,dataset
7,23,Some remarks on selective migration,NaN,"Hofstee, E.W.","Hofstee, E.W.",NaN,NaN,1952,1952,article,NaN,NaN,NaN,1952,1952,7,Publications of the research group for europea...,Staatsdrukkerij,main,REMP_IM
43,67,Possibilities for inter-European Migration fro...,NaN,"Hofstee, E.W. && Groenman, Sj.","Hofstee, E.W. && Groenman, Sj.",&&,NaN,1953,1953,article,NaN,NaN,NaN,1953,1953,vol.1,REMP-bulletin,Staatsdrukkerij,main,REMP_IM
51,75,Avantages et inconvenients de l'emigration ext...,NaN,"Brink, van den, T && Hofstee, E.W.","Brink, van den, T && Hofstee, E.W.",&&,NaN,1954,1954,article,NaN,NaN,NaN,1954,1954,vol.2,REMP-bulletin,Staatsdrukkerij,main,REMP_IM
63,87,Some preliminary conclusions concerning intern...,NaN,"Hofstee, E.W.","Hofstee, E.W.",NaN,NaN,1954,1954,article,NaN,NaN,NaN,1954,1954,vol.2,REMP-bulletin,Staatsdrukkerij,main,REMP_IM
137,165,Thwarted exodus : post-war overseas migration ...,NaN,"Hofstede, B.P.","Hofstede, B.P.",NaN,NaN,1964,1964,article,NaN,NaN,NaN,1964,1964,9,Studies in Social Life,Staatsdrukkerij,main,REMP_IM
410,401,On Anticipating The Future,https://onlinelibrary.wiley.com/doi/10.1111/j....,D. H. Hofmeije,"Hofmeije, D H",NaN,156-164,01 July 1966,1966,Original Article,3-4,"International Migration: Vol 4, No 3‐4",139-215,July 1966,1966,4,International Migration,Wiley,main,REMP_IM
1648,853,4: A Sociological Approach toward a General Th...,https://journals.sagepub.com/doi/pdf/10.1177/0...,Hans-Joachim Hoffmann-Nowotny,"Hoffmann-Nowotny, Hans-Joachim",NaN,64-83,"Jan 1, 1981",1981,Article,1_suppl,"International Migration Review - Volume 15, Nu...",NaN,"Jan 01, 1981",1981,15,International Migration Review,Sage Publishing,main,IMR_research
2546,1886,Book Review: Berufsvorbereitende und beruflich...,https://journals.sagepub.com/doi/pdf/10.1177/0...,Hans-Joachim Hoffmann-Nowotny,"Hoffmann-Nowotny, Hans-Joachim",Sociological Institute of the University of Zu...,181-181,"Mar 1, 1987",1987,Book Reviews,1,"International Migration Review - Volume 21, Nu...",NaN,"Mar 01, 1987",1987,21,International Migration Review,Sage Publishing,review,IMR_review
4155,3770,Language Shift and Maintenance in Israel,https://journals.sagepub.com/doi/pdf/10.1177/0...,John E. Hofman && Haya Fisherman,"Hofman, John E && Fisherman, Haya","Haifa University College; Haya Fisherman, The ...",204-226,"Jun 1, 1971",1971,Article,2,"International Migration Review - Volume 5, Num...",NaN,"Jun 01, 1971",1971,5,International Migration Review,Sage Publishing,main,IMR_research
4265,3892,12 Switzerland: Remaining Swiss,https://journals.sagepub.com/doi/pdf/10.1177/0...,Hans-joachim Hoffmann-nowotny && Martin Killias,"Hoffmann-nowotny, Hans-joachim && Killias, Martin",University of Zuerich && University of Lausanne,231-245,"Jan 1, 1993",1993,Article,1_suppl,"International Migration Review - Volume 27, Nu...",NaN,"Jan 01, 1993",1993,27,International Migration Review,Sage Publishing,main,IMR_research


In [6]:
temp_df = pub_df[pub_df.article_author_index_name.isna() == False]
temp_df[temp_df.article_author_index_name.str.contains('Cnossen')]
temp_df[temp_df.article_author_index_name.str.contains(' Por ')]


,Unnamed: 0,article_title,article_doi,article_author,article_author_index_name,article_author_affiliation,article_page_range,article_pub_date,article_pub_year,issue_section,issue_number,issue_title,issue_page_range,issue_pub_date,issue_pub_year,volume,journal,publisher,article_type,dataset
580,637,Participación de los inmigrantes en la vida de...,https://onlinelibrary.wiley.com/doi/10.1111/j....,Por Jerzy Zubrzycki,"Zubrzycki, Por Jerzy",NaN,181-206,01 July 1964,1964,Original Article,3,"International Migration: Vol 2, No 3",179-251,July 1964,1964,2,International Migration,Wiley,main,REMP_IM
884,1144,"Latinoamericanos en Europa, Aspectos Demogràfi...",https://onlinelibrary.wiley.com/doi/10.1111/j....,Por Julio Morales Vergara,"Vergara, Por Julio Morales",NaN,14-32,01 January 1974,1974,Original Article,1-2,"International Migration: Vol 12, No 1‐2",5-104,January 1974,1974,12,International Migration,Wiley,main,REMP_IM


In [7]:
temp_df = pub_df[pub_df.issue_title.isna() == False]

temp_df[temp_df.issue_title.str.contains('International Migration: Vol 2, No 3')]

,Unnamed: 0,article_title,article_doi,article_author,article_author_index_name,article_author_affiliation,article_page_range,article_pub_date,article_pub_year,issue_section,issue_number,issue_title,issue_page_range,issue_pub_date,issue_pub_year,volume,journal,publisher,article_type,dataset
580,637,Participación de los inmigrantes en la vida de...,https://onlinelibrary.wiley.com/doi/10.1111/j....,Por Jerzy Zubrzycki,"Zubrzycki, Por Jerzy",NaN,181-206,01 July 1964,1964,Original Article,3,"International Migration: Vol 2, No 3",179-251,July 1964,1964,2,International Migration,Wiley,main,REMP_IM
581,638,La actitud de los dirigentes de grupos de inmi...,https://onlinelibrary.wiley.com/doi/10.1111/j....,D. J. Lawless,"Lawless, D J",NaN,207-218,01 July 1964,1964,Original Article,3,"International Migration: Vol 2, No 3",179-251,July 1964,1964,2,International Migration,Wiley,main,REMP_IM
582,640,Thwarted Exodus,https://onlinelibrary.wiley.com/doi/10.1111/j....,Comentario Critico De T. Cnossen,"Cnossen, de T",NaN,224-227,01 July 1964,1964,Original Article,3,"International Migration: Vol 2, No 3",179-251,July 1964,1964,2,International Migration,Wiley,main,REMP_IM
583,641,Tendencias De La Inmigración En El Canadá,https://onlinelibrary.wiley.com/doi/10.1111/j....,POR G. D. McQUADE,"McQUADE, G D",NaN,228-241,01 July 1964,1964,Original Article,3,"International Migration: Vol 2, No 3",179-251,July 1964,1964,2,International Migration,Wiley,main,REMP_IM


## Clustering Author Names

We want to see which authors published in both journals, and how often. This requires a number of transformations:

1. splitting records of multi-author papers into a record per author
2. normalising author names such that variant spellings are mapped to a single version. 

The latter step is always a risky operation, because using only the surface form of a name can results in two persons with similar names being considered as a single person. Given that this dataset narrowly focuses in only authors of articles in the two journals, we assume the chance that two authors have the same surname and initials is low. 


#### Splitting multi-author records

In [8]:
# Code adapted from https://stackoverflow.com/questions/50731229/split-cell-into-multiple-rows-in-pandas-dataframe

import numpy as np
from itertools import chain

# return list from series of comma-separated strings
def chainer(s):
    return list(chain.from_iterable(s.fillna('').str.split(' && ')))

# calculate lengths of splits
lens = pub_df['article_author'].fillna('').str.split(' && ').map(len)

# create new dataframe, repeating or chaining as appropriate
split_pub_df = pd.DataFrame({
    'journal': np.repeat(pub_df['journal'], lens),
    'issue_pub_year': np.repeat(pub_df['issue_pub_year'], lens),
    'publisher': np.repeat(pub_df['publisher'], lens),
    'dataset': np.repeat(pub_df['dataset'], lens),
    'article_author': chainer(pub_df['article_author']),
    'article_author_index_name': chainer(pub_df['article_author_index_name']),
    'article_author_affiliation': chainer(pub_df['article_author_affiliation'])
})

split_pub_df = split_pub_df.reset_index(drop=True)
split_pub_df

,journal,issue_pub_year,publisher,dataset,article_author,article_author_index_name,article_author_affiliation
0,Publications of the research group for europea...,1951,Staatsdrukkerij,REMP_IM,"Wander, H.","Wander, H.",
1,Publications of the research group for europea...,1951,Staatsdrukkerij,REMP_IM,"Citroen, H.A.","Citroen, H.A.",
2,Publications of the research group for europea...,1952,Staatsdrukkerij,REMP_IM,"Beijer, G.","Beijer, G.",
3,Publications of the research group for europea...,1952,Staatsdrukkerij,REMP_IM,"Oudegeest, J.J.","Oudegeest, J.J.",
4,Publications of the research group for europea...,1952,Staatsdrukkerij,REMP_IM,"Brink, van den, T.","Brink, van den, T.",
...,...,...,...,...,...,...,...
4827,International Migration Review,1986,Sage Publishing,IMR_research,,,
4828,International Migration Review,1986,Sage Publishing,IMR_research,,,
4829,International Migration Review,1986,Sage Publishing,IMR_research,,,
4830,International Migration Review,1986,Sage Publishing,IMR_research,,,


In [9]:
split_pub_df[split_pub_df.article_author.str.contains('Avila')]
split_pub_df[split_pub_df.article_author.str.contains('\u00c1vila')]
split_pub_df[split_pub_df.article_author_index_name.str.contains('Charr')]
#split_pub_df[split_pub_df.article_author_index_name.str.contains('Clark')]
#split_pub_df[split_pub_df.article_author_index_name.str.contains('Daley')]


,journal,issue_pub_year,publisher,dataset,article_author,article_author_index_name,article_author_affiliation
663,International Migration,1992,Wiley,REMP_IM,Ambassador Héctor Charry-Samper,"Charry-Samper, Ambassador Héctor",


#### Normalising author names

There is a lot of variation in how author names are represented. Sometimes with full first and middle names, sometime with only the first name or only initials, or the first name in full but the middle names as initials.

We start from the author format where the surname is followed by the first and middle names (field `article_author_index_name`). We apply the following normalisation and mapping steps:

1. transform the `article_author_index_name` to title casing (meaning each initial character of a name part is uppercase and the rest is lowercase),
2. remove everything after the first letter that follows the surname,
3. transform all uses of `ij` to `y` as this Dutch and German names containing `ij` are sometimes spelled with `y`, e.g. `Gunther Beijer` vs. `Gunther Beyer`.


In [10]:
import re 
import unicodedata
import string

all_letters = string.ascii_letters + " .,;'-"

# Turn a Unicode string to plain ASCII, thanks to https://stackoverflow.com/a/518232/2809427
def unicode_to_ascii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
        and c in all_letters
    )

def acronym(string):
    if string == 'International Migration':
        return 'IM'
    else:
        return 'IMR'


def normalise_name(author_name):
    author_name = unicode_to_ascii(author_name)
    author_name = author_name.title()
    author_name = author_name.replace('Abandan-Unat', 'Abadan-Unat')
    author_name = author_name.replace('Bastos De Avila', 'Avila')
    author_name = author_name.replace('Purcell Jr.', 'Purcell') # Jr. is dropped from the index name
    # Titles like Father
    titles = ['Father ', 'Ambassador ']
    for title in titles:
        if title in author_name:
            author_name = author_name.replace(title, ' ')
    # Prefix in Spanish and French: Por and Par
    if ' Por ' in author_name:
        author_name = author_name.replace(' Por ', ' ')
    if ' Par ' in author_name:
        author_name = author_name.replace(' Par ', ' ')
    author_name = re.sub(r' +',' ', author_name)
    return author_name.strip()


def parse_surname(author_name: str):
    author_name = normalise_name(author_name)
    return ','.join(author_name.split(',')[:-1]).replace('ij', 'y').title()


def parse_surname_initial(author_name: str):
    author_name = normalise_name(author_name)
    if ',' not in author_name:
        return author_name
    surname = ','.join(author_name.split(',')[:-1]).replace('ij', 'y').title()
    initial = author_name.split(', ')[-1][0]
    return f'{surname}, {initial}'

unicode_to_ascii('\u00c1vila')
unicode_to_ascii('Charry-Samper, Ambassador Héctor')
parse_surname_initial('Charry-Samper, Ambassador Héctor')

'Charry-Samper, H'

In [11]:

split_pub_df['article_author_index_name'] = split_pub_df['article_author_index_name'].str.title()
split_pub_df['author_surname_initial'] = split_pub_df.article_author_index_name.apply(parse_surname_initial)
split_pub_df['author_surname'] = split_pub_df.article_author_index_name.apply(parse_surname)
split_pub_df['issue_pub_decade'] = split_pub_df.issue_pub_year.apply(lambda x: int(x/10)*10)

split_pub_df.journal = split_pub_df.journal.apply(acronym)

split_pub_df

,journal,issue_pub_year,publisher,dataset,article_author,article_author_index_name,article_author_affiliation,author_surname_initial,author_surname,issue_pub_decade
0,IMR,1951,Staatsdrukkerij,REMP_IM,"Wander, H.","Wander, H.",,"Wander, H",Wander,1950
1,IMR,1951,Staatsdrukkerij,REMP_IM,"Citroen, H.A.","Citroen, H.A.",,"Citroen, H",Citroen,1950
2,IMR,1952,Staatsdrukkerij,REMP_IM,"Beijer, G.","Beijer, G.",,"Beyer, G",Beyer,1950
3,IMR,1952,Staatsdrukkerij,REMP_IM,"Oudegeest, J.J.","Oudegeest, J.J.",,"Oudegeest, J",Oudegeest,1950
4,IMR,1952,Staatsdrukkerij,REMP_IM,"Brink, van den, T.","Brink, Van Den, T.",,"Brink, Van Den, T","Brink, Van Den",1950
...,...,...,...,...,...,...,...,...,...,...
4827,IMR,1986,Sage Publishing,IMR_research,,,,,,1980
4828,IMR,1986,Sage Publishing,IMR_research,,,,,,1980
4829,IMR,1986,Sage Publishing,IMR_research,,,,,,1980
4830,IMR,1986,Sage Publishing,IMR_research,,,,,,1980


In [12]:
split_pub_df[split_pub_df.author_surname_initial.str.contains('Charry')]

,journal,issue_pub_year,publisher,dataset,article_author,article_author_index_name,article_author_affiliation,author_surname_initial,author_surname,issue_pub_decade
663,IM,1992,Wiley,REMP_IM,Ambassador Héctor Charry-Samper,"Charry-Samper, Ambassador Héctor",,"Charry-Samper, H",Charry-Samper,1990


In [13]:
# remove articles with no authors
split_pub_df =  split_pub_df[split_pub_df.article_author != '']
split_pub_df

,journal,issue_pub_year,publisher,dataset,article_author,article_author_index_name,article_author_affiliation,author_surname_initial,author_surname,issue_pub_decade
0,IMR,1951,Staatsdrukkerij,REMP_IM,"Wander, H.","Wander, H.",,"Wander, H",Wander,1950
1,IMR,1951,Staatsdrukkerij,REMP_IM,"Citroen, H.A.","Citroen, H.A.",,"Citroen, H",Citroen,1950
2,IMR,1952,Staatsdrukkerij,REMP_IM,"Beijer, G.","Beijer, G.",,"Beyer, G",Beyer,1950
3,IMR,1952,Staatsdrukkerij,REMP_IM,"Oudegeest, J.J.","Oudegeest, J.J.",,"Oudegeest, J",Oudegeest,1950
4,IMR,1952,Staatsdrukkerij,REMP_IM,"Brink, van den, T.","Brink, Van Den, T.",,"Brink, Van Den, T","Brink, Van Den",1950
...,...,...,...,...,...,...,...,...,...,...
4817,IMR,1993,Sage Publishing,IMR_research,Silva Meznaric,"Meznaric, Silva",Institute for Migration and Ethnic Studies Uni...,"Meznaric, S",Meznaric,1990
4818,IMR,1993,Sage Publishing,IMR_research,Jadranka Caci-kumpes,"Caci-Kumpes, Jadranka",Institute for Migration and Ethnic Studies Uni...,"Caci-Kumpes, J",Caci-Kumpes,1990
4819,IMR,1993,Sage Publishing,IMR_research,Masatoshi Muto,"Muto, Masatoshi","Ministry of Foreign Affairs, Tokyo","Muto, M",Muto,1990
4820,IMR,1993,Sage Publishing,IMR_research,Lilia Shevtsova,"Shevtsova, Lilia","USSR Academy of Sciences, Moscow","Shevtsova, L",Shevtsova,1990


In [14]:
temp_df = split_pub_df[['author_surname', 'author_surname_initial']].drop_duplicates()
counts = temp_df.author_surname.value_counts()
for row in temp_df[temp_df.author_surname.apply(lambda x: counts[x] > 2)].sort_values('author_surname').iterrows():
    print(row[1]['author_surname'], row[1]['author_surname_initial'])

 Remp
 Ministry Of Labour Japan
 Migrator
 D.K.
 J.Z.
 Unaids And Iom
 G.C.S.
 A.H.R.
Abraham Abraham, S
Abraham Abraham, M
Abraham Abraham, N
Anderson Anderson, B
Anderson Anderson, P
Anderson Anderson, A
Bailey Bailey, A
Bailey Bailey, S
Bailey Bailey, T
Baker Baker, S
Baker Baker, D
Baker Baker, P
Bennett Bennett, N
Bennett Bennett, A
Bennett Bennett, S
Bernard Bernard, J
Bernard Bernard, R
Bernard Bernard, T
Bernard Bernard, W
Brown Brown, D
Brown Brown, L
Brown Brown, M
Chin Chin, R
Chin Chin, K
Chin Chin, C
Ellis Ellis, R
Ellis Ellis, J
Ellis Ellis, M
Evans Evans, M
Evans Evans, J
Evans Evans, G
Fong Fong, H
Fong Fong, E
Fong Fong, T
Fong Fong, P
Garcia Garcia, P
Garcia Garcia, F
Garcia Garcia, M
Garcia Garcia, J
Garcia Garcia, C
Gerber Gerber, D
Gerber Gerber, S
Gerber Gerber, R
Gonzalez Gonzalez, J
Gonzalez Gonzalez, N
Gonzalez Gonzalez, L
Gonzalez Gonzalez, G
Green Green, S
Green Green, V
Green Green, R
Green Green, A
Griffith Griffith, D
Griffith Griffith, E
Griffith Griffith

In [15]:
split_pub_df[split_pub_df.author_surname == 'Mast, Van Der']
split_pub_df[split_pub_df.author_surname == 'Haveman']
split_pub_df[split_pub_df.author_surname == 'Avila']
split_pub_df[split_pub_df.author_surname.str.contains('Wink')]

,journal,issue_pub_year,publisher,dataset,article_author,article_author_index_name,article_author_affiliation,author_surname_initial,author_surname,issue_pub_decade
2631,IMR,1981,Sage Publishing,IMR_research,Elizabeth Winkler,"Winkler, Elizabeth",,"Winkler, E",Winkler,1980


In [16]:
temp_df = split_pub_df[['publisher', 'author_surname_initial']]
g = temp_df.groupby(['publisher', 'author_surname_initial']).size()
au_pub_df = g.unstack('publisher').fillna(0.0)
au_pub_df

publisher,Sage Publishing,Staatsdrukkerij,Wiley
author_surname_initial,,,
A.H.R.,1.0,0.0,0.0
"Abad, R",2.0,0.0,0.0
"Abadan-Unat, N",6.0,0.0,0.0
"Abalos, D",1.0,0.0,0.0
"Abell, N",1.0,0.0,0.0
...,...,...,...
"Zodgekar, A",0.0,0.0,1.0
"Zolberg, A",4.0,0.0,0.0
"Zubrzycki, J",3.0,3.0,6.0


In [17]:
temp_df = split_pub_df[['publisher', 'author_surname_initial','issue_pub_decade']]
g = temp_df.groupby(['publisher', 'author_surname_initial','issue_pub_decade']).size()
decade_au_pub_df = g.unstack(['publisher','issue_pub_decade']).fillna(0.0)
decade_au_pub_df=decade_au_pub_df.reindex([1950,1960,1970,1980], axis=1, level=1)
decade_au_pub_df

publisher              Sage Publishing           Staatsdrukkerij            \
issue_pub_decade                  1960 1970 1980            1950 1960 1970   
author_surname_initial                                                       
A.H.R.                             0.0  1.0  0.0             0.0  0.0  0.0   
Abad, R                            0.0  0.0  2.0             0.0  0.0  0.0   
Abadan-Unat, N                     0.0  1.0  2.0             0.0  0.0  0.0   
Abalos, D                          0.0  1.0  0.0             0.0  0.0  0.0   
Abell, N                           0.0  0.0  0.0             0.0  0.0  0.0   
...                                ...  ...  ...             ...  ...  ...   
Zodgekar, A                        0.0  0.0  0.0             0.0  0.0  0.0   
Zolberg, A                         0.0  0.0  4.0             0.0  0.0  0.0   
Zubrzycki, J                       0.0  2.0  1.0             3.0  0.0  0.0   
Zucchi, J                          0.0  0.0  1.0             0.0  0.0  0.0   
Zucker, N                          0.0  0.0  0.0             0.0  0.0  0.0   

publisher                   Wiley            
issue_pub_decade       1980  1960 1970 1980  
author_surname_initial                       
A.H.R.                  0.0   0.0  0.0  0.0  
Abad, R                 0.0   0.0  0.0  0.0  
Abadan-Unat, N          0.0   0.0  0.0  0.0  
Abalos, D               0.0   0.0  0.0  0.0  
Abell, N                0.0   0.0  0.0  0.0  
...                     ...   ...  ...  ...  
Zodgekar, A             0.0   0.0  0.0  0.0  
Zolberg, A              0.0   0.0  0.0  0.0  
Zubrzycki, J            0.0   5.0  0.0  1.0  
Zucchi, J               0.0   0.0  0.0  0.0  
Zucker, N               0.0   0.0  0.0  0.0  

[2498 rows x 10 columns]

In [18]:
au_pub_df.sort_values(by=['Staatsdrukkerij', 'Sage Publishing', 'Wiley'], ascending=False)

publisher,Sage Publishing,Staatsdrukkerij,Wiley
author_surname_initial,,,
"Beyer, G",3.0,12.0,9.0
"Edding, F",0.0,4.0,0.0
"Hofstee, E",0.0,4.0,0.0
"Zubrzycki, J",3.0,3.0,6.0
"Richardson, A",3.0,3.0,2.0
...,...,...,...
"Zagorski, K",0.0,0.0,1.0
"Zammit, H",0.0,0.0,1.0
"Zanartu, M",0.0,0.0,1.0


In [19]:
split_pub_df

,journal,issue_pub_year,publisher,dataset,article_author,article_author_index_name,article_author_affiliation,author_surname_initial,author_surname,issue_pub_decade
0,IMR,1951,Staatsdrukkerij,REMP_IM,"Wander, H.","Wander, H.",,"Wander, H",Wander,1950
1,IMR,1951,Staatsdrukkerij,REMP_IM,"Citroen, H.A.","Citroen, H.A.",,"Citroen, H",Citroen,1950
2,IMR,1952,Staatsdrukkerij,REMP_IM,"Beijer, G.","Beijer, G.",,"Beyer, G",Beyer,1950
3,IMR,1952,Staatsdrukkerij,REMP_IM,"Oudegeest, J.J.","Oudegeest, J.J.",,"Oudegeest, J",Oudegeest,1950
4,IMR,1952,Staatsdrukkerij,REMP_IM,"Brink, van den, T.","Brink, Van Den, T.",,"Brink, Van Den, T","Brink, Van Den",1950
...,...,...,...,...,...,...,...,...,...,...
4817,IMR,1993,Sage Publishing,IMR_research,Silva Meznaric,"Meznaric, Silva",Institute for Migration and Ethnic Studies Uni...,"Meznaric, S",Meznaric,1990
4818,IMR,1993,Sage Publishing,IMR_research,Jadranka Caci-kumpes,"Caci-Kumpes, Jadranka",Institute for Migration and Ethnic Studies Uni...,"Caci-Kumpes, J",Caci-Kumpes,1990
4819,IMR,1993,Sage Publishing,IMR_research,Masatoshi Muto,"Muto, Masatoshi","Ministry of Foreign Affairs, Tokyo","Muto, M",Muto,1990
4820,IMR,1993,Sage Publishing,IMR_research,Lilia Shevtsova,"Shevtsova, Lilia","USSR Academy of Sciences, Moscow","Shevtsova, L",Shevtsova,1990


## Parsing Person Records

In [88]:
from scripts.network_analysis import retrieve_spreadsheet_records

entity_records = retrieve_spreadsheet_records(record_type='categories')
print('Number of records:' , len(entity_records))


Number of records: 74


In [89]:
import json

for record in entity_records:
    print(json.dumps(record, indent=4))
    #print(record['Organisation'], record['Prs_surname'])

{
    "organisation": "REMP",
    "period_start": "1952",
    "last_known_date": "1983",
    "prs_id": "1",
    "prs_surname": "Beijer",
    "prs_infix": "",
    "prs_initials": "G.",
    "prs_function": "demographer, The Hague",
    "prs_category": "academic",
    "is_academic": "yes",
    "is_public_administration": "",
    "prs_country": "NL",
    "prs_role1": "founder",
    "prs_role2": "member_MC",
    "prs_role3": "secretary-editor",
    "remarks": "director-editor (1969)"
}
{
    "organisation": "REMP",
    "period_start": "1952",
    "last_known_date": "1969",
    "prs_id": "2",
    "prs_surname": "Groenman",
    "prs_infix": "",
    "prs_initials": "Sj.",
    "prs_function": "sociologist, Leiden",
    "prs_category": "academic",
    "is_academic": "1947",
    "is_public_administration": "1943-1950",
    "prs_country": "NL",
    "prs_role1": "founder",
    "prs_role2": "member_MC",
    "prs_role3": "vice-chair_BoD",
    "remarks": ""
}
{
    "organisation": "REMP",
    "period_

In [90]:
import pandas as pd

def parse_author_index_name(row):
    if row['prs_infix'] != '':
        return ', '.join([row['prs_surname'], row['prs_infix'], row['prs_initials']])
    else:
        return ', '.join([row['prs_surname'], row['prs_initials']])
    
board_df = pd.DataFrame(entity_records)
b_cols = {c:c.lower() for c in board_df.columns}
board_df.rename(columns=b_cols, inplace=True)
board_df['article_author_index_name'] = board_df.apply(parse_author_index_name, axis=1)
board_df['author_surname_initial'] = board_df.article_author_index_name.apply(parse_surname_initial)
board_df

,organisation,period_start,last_known_date,prs_id,prs_surname,prs_infix,prs_initials,prs_function,prs_category,is_academic,is_public_administration,prs_country,prs_role1,prs_role2,prs_role3,remarks,article_author_index_name,author_surname_initial
0,REMP,1952,1983,1,Beijer,,G.,demographe...,academic,yes,,NL,founder,member_MC,secretary-...,director-e...,"Beijer, G.","Beyer, G"
1,REMP,1952,1969,2,Groenman,,Sj.,sociologis...,academic,1947,1943-1950,NL,founder,member_MC,vice-chair...,,"Groenman, Sj.","Groenman, S"
2,REMP,1952,1969,3,Zeegers,,G.H.L.,"economist,...",academic,yes,1941-1950,NL,founder,member_MC,member_BoD,,"Zeegers, G...","Zeegers, G"
3,REMP,1952,1969,4,Hofstee,,E.W.,sociologis...,academic,yes,"yes, advis...",NL,founder,member_BoD,,,"Hofstee, E.W.","Hofstee, E"
4,REMP,1952,1969,5,Bouman,,P.J.,sociologis...,academic,yes,,NL,member_BoD,,chair_BoD ...,,"Bouman, P.J.","Bouman, P"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69,ICEM,1970,1988,68,Maselli,,G.,deputy dir...,,,,IT,,,,,"Maselli, G.","Maselli, G"
70,ICEM,1989,1993,69,Charry-Samper,,H.,deputy dir...,,,,CO,,,,,Charry-Sam...,Charry-Sam...
71,ICEM,1994,1999,70,Escaler,,N.L. (Narc...,deputy dir...,,,,PH,,,,,"Escaler, N...","Escaler, N"
72,ICEM,1999,2009,71,Ndioro,,N. (Ndiaye),deputy dir...,,,,SN,,,,,"Ndioro, N....","Ndioro, N"


In [91]:
board_df[board_df.author_surname_initial.str.contains('Purc')]

,organisation,period_start,last_known_date,prs_id,prs_surname,prs_infix,prs_initials,prs_function,prs_category,is_academic,is_public_administration,prs_country,prs_role1,prs_role2,prs_role3,remarks,article_author_index_name,author_surname_initial
61,ICEM,1988,1998,61,Purcell jr.,,James,director g...,,,,USA,,,,,Purcell jr...,"Purcell, J"


In [92]:
temp_df = board_df[['author_surname_initial', 'organisation']]
g = temp_df.groupby(['author_surname_initial', 'organisation']).size()
org_df = g.unstack(['organisation']).fillna(0.0)
org_df


organisation,Dutch Government,ICEM,REMP
author_surname_initial,,,
"Appleyard, R",0.0,0.0,1.0
"Avila, F",0.0,0.0,1.0
"Baade, F",0.0,0.0,1.0
"Backer, J",0.0,0.0,1.0
"Besterman, W",0.0,1.0,0.0
...,...,...,...
"Vito, F",0.0,0.0,1.0
"Vitorino, A",0.0,1.0,0.0
"Weinberg, A",0.0,0.0,1.0


can we give the board dataframe a temporal element. First try - include it in columns (but how to combine this with the title dataframe)


In [93]:
def yr2cat(x):
    s = x['period_start']
    e = x['last_known_date']
    try:
        start = int(s)
    except ValueError:
        start = 0
    try:
        end = int(e)
    except ValueError:
        end = start
    return pd.Interval(start, end, closed='both')

In [94]:
board_df['period'] = board_df.apply(lambda x: yr2cat(x), axis=1)

In [95]:
decades = {1950:(1950, 1960),
 1960:(1960, 1970),
 1970:(1970, 1980),
 1980:(1980, 1990),
 1990:(1990, 2000),
 2000:(2000, 2010)
          }

def cutdecade(x, decade):
    result = False
    if x.right < decade[0]:
        return False
    if x.left > decade[1]:
        return False
    if x.left > decade[0] or x.right >= decade[0]:
        return True



for key in decades:
    decade = decades[key]
    board_df[str(key)] = board_df.period.apply(lambda x: cutdecade(x, decade))

In [96]:
boarders = board_df.author_surname_initial.to_frame()
boarders

,author_surname_initial
0,"Beyer, G"
1,"Groenman, S"
2,"Zeegers, G"
3,"Hofstee, E"
4,"Bouman, P"
...,...
69,"Maselli, G"
70,Charry-Sam...
71,"Escaler, N"
72,"Ndioro, N"


In [37]:
def trueto1(x):
    if x ==True:
        return 1
    else:
        return 0

In [97]:
for board in board_df.organisation.unique():
    for year in '1950','1960', '1970', '1980', '1990', '2000':
        selection = board_df.loc[(board_df['organisation']==board)&(board_df[year]==True)]['author_surname_initial']
        boarders[f'{board}_{year}'] = board_df.author_surname_initial.isin(selection).apply(lambda x: trueto1(x))

In [98]:

au_org_df = pd.merge(au_pub_df.reset_index(), org_df.reset_index(), on='author_surname_initial', how='left').fillna(0.0)

au_org_df



,author_surname_initial,Sage Publishing,Staatsdrukkerij,Wiley,Dutch Government,ICEM,REMP
0,A.H.R.,1.0,0.0,0.0,0.0,0.0,0.0
1,"Abad, R",2.0,0.0,0.0,0.0,0.0,0.0
2,Abadan-Una...,6.0,0.0,0.0,0.0,0.0,0.0
3,"Abalos, D",1.0,0.0,0.0,0.0,0.0,0.0
4,"Abell, N",1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...
2493,"Zodgekar, A",0.0,0.0,1.0,0.0,0.0,0.0
2494,"Zolberg, A",4.0,0.0,0.0,0.0,0.0,0.0
2495,"Zubrzycki, J",3.0,3.0,6.0,0.0,0.0,0.0
2496,"Zucchi, J",1.0,0.0,0.0,0.0,0.0,0.0


In [99]:
au_org_df2 = pd.merge(left=au_pub_df, right=boarders, on='author_surname_initial', how='left').fillna(0)
au_org_df2

,author_surname_initial,Sage Publishing,Staatsdrukkerij,Wiley,REMP_1950,REMP_1960,REMP_1970,REMP_1980,REMP_1990,REMP_2000,...,Dutch Government_1970,Dutch Government_1980,Dutch Government_1990,Dutch Government_2000,ICEM_1950,ICEM_1960,ICEM_1970,ICEM_1980,ICEM_1990,ICEM_2000
0,A.H.R.,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,"Abad, R",2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Abadan-Una...,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,"Abalos, D",1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,"Abell, N",1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2494,"Zodgekar, A",0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2495,"Zolberg, A",4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2496,"Zubrzycki, J",3.0,3.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2497,"Zucchi, J",1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [100]:
decade_au_org_df = pd.merge(left=decade_au_pub_df, right=boarders, on='author_surname_initial', how='left').fillna(0)
decade_au_org_df.drop_duplicates(inplace=True)
decade_au_org_df.rename(columns={c:f"{c[0]}_{c[1]}" for c in decade_au_org_df.columns if type(c[1])==int }, inplace=True)

/Users/rikhoekstra/anaconda/envs/36/lib/python3.7/site-packages/pandas/core/reshape/merge.py:643: UserWarning: merging between different levels can give an unintended result (2 levels on the left,1 on the right)
  warnings.warn(msg, UserWarning)


In [101]:
colranges = {}
for i in range(1950,2010,10):
    colranges[f"{i}"] = [c for c in decade_au_org_df.columns if f"{i}" in c]

In [102]:
colranges

{'1950': ['Staatsdrukkerij_1950',
  'REMP_1950',
  'Dutch Government_1950',
  'ICEM_1950'],
 '1960': ['Sage Publishing_1960',
  'Staatsdrukkerij_1960',
  'Wiley_1960',
  'REMP_1960',
  'Dutch Government_1960',
  'ICEM_1960'],
 '1970': ['Sage Publishing_1970',
  'Staatsdrukkerij_1970',
  'Wiley_1970',
  'REMP_1970',
  'Dutch Government_1970',
  'ICEM_1970'],
 '1980': ['Sage Publishing_1980',
  'Staatsdrukkerij_1980',
  'Wiley_1980',
  'REMP_1980',
  'Dutch Government_1980',
  'ICEM_1980'],
 '1990': ['REMP_1990', 'Dutch Government_1990', 'ICEM_1990'],
 '2000': ['REMP_2000', 'Dutch Government_2000', 'ICEM_2000']}

In [103]:
colorder=[]
for c in colranges.values():
    colorder.extend(c)
colorder

['Staatsdrukkerij_1950',
 'REMP_1950',
 'Dutch Government_1950',
 'ICEM_1950',
 'Sage Publishing_1960',
 'Staatsdrukkerij_1960',
 'Wiley_1960',
 'REMP_1960',
 'Dutch Government_1960',
 'ICEM_1960',
 'Sage Publishing_1970',
 'Staatsdrukkerij_1970',
 'Wiley_1970',
 'REMP_1970',
 'Dutch Government_1970',
 'ICEM_1970',
 'Sage Publishing_1980',
 'Staatsdrukkerij_1980',
 'Wiley_1980',
 'REMP_1980',
 'Dutch Government_1980',
 'ICEM_1980',
 'REMP_1990',
 'Dutch Government_1990',
 'ICEM_1990',
 'REMP_2000',
 'Dutch Government_2000',
 'ICEM_2000']

In [104]:
decade_au_org_df.loc[decade_au_org_df.author_surname_initial.str.contains('Haveman')]

,author_surname_initial,Sage Publishing_1960,Sage Publishing_1970,Sage Publishing_1980,Staatsdrukkerij_1950,Staatsdrukkerij_1960,Staatsdrukkerij_1970,Staatsdrukkerij_1980,Wiley_1960,Wiley_1970,...,Dutch Government_1970,Dutch Government_1980,Dutch Government_1990,Dutch Government_2000,ICEM_1950,ICEM_1960,ICEM_1970,ICEM_1980,ICEM_1990,ICEM_2000
918,"Haveman, B",0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [106]:
decade_au_org_df.columns

Index(['author_surname_initial', 'Sage Publishing_1960',
       'Sage Publishing_1970', 'Sage Publishing_1980', 'Staatsdrukkerij_1950',
       'Staatsdrukkerij_1960', 'Staatsdrukkerij_1970', 'Staatsdrukkerij_1980',
       'Wiley_1960', 'Wiley_1970', 'Wiley_1980', 'REMP_1950', 'REMP_1960',
       'REMP_1970', 'REMP_1980', 'REMP_1990', 'REMP_2000',
       'Dutch Government_1950', 'Dutch Government_1960',
       'Dutch Government_1970', 'Dutch Government_1980',
       'Dutch Government_1990', 'Dutch Government_2000', 'ICEM_1950',
       'ICEM_1960', 'ICEM_1970', 'ICEM_1980', 'ICEM_1990', 'ICEM_2000'],
      dtype='object')

In [107]:
au_org_df2.loc[au_org_df2.author_surname_initial.str.contains('Haveman')]

,author_surname_initial,Sage Publishing,Staatsdrukkerij,Wiley,REMP_1950,REMP_1960,REMP_1970,REMP_1980,REMP_1990,REMP_2000,...,Dutch Government_1970,Dutch Government_1980,Dutch Government_1990,Dutch Government_2000,ICEM_1950,ICEM_1960,ICEM_1970,ICEM_1980,ICEM_1990,ICEM_2000
918,"Haveman, B",0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
919,"Haveman, B",0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [108]:
au_org_df

,author_surname_initial,Sage Publishing,Staatsdrukkerij,Wiley,Dutch Government,ICEM,REMP
0,A.H.R.,1.0,0.0,0.0,0.0,0.0,0.0
1,"Abad, R",2.0,0.0,0.0,0.0,0.0,0.0
2,Abadan-Una...,6.0,0.0,0.0,0.0,0.0,0.0
3,"Abalos, D",1.0,0.0,0.0,0.0,0.0,0.0
4,"Abell, N",1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...
2493,"Zodgekar, A",0.0,0.0,1.0,0.0,0.0,0.0
2494,"Zolberg, A",4.0,0.0,0.0,0.0,0.0,0.0
2495,"Zubrzycki, J",3.0,3.0,6.0,0.0,0.0,0.0
2496,"Zucchi, J",1.0,0.0,0.0,0.0,0.0,0.0


In [109]:

org_au_df = pd.merge(au_pub_df.reset_index(), org_df.reset_index(), on='author_surname_initial', how='right').fillna(0.0)

org_au_df.sort_values(by=['Dutch Government'], ascending=False)



,author_surname_initial,Sage Publishing,Staatsdrukkerij,Wiley,Dutch Government,ICEM,REMP
34,"Klompe, M",0.0,0.0,0.0,1.0,0.0,0.0
66,Verwey-Jon...,0.0,1.0,0.0,1.0,0.0,0.0
24,"Haveman, B",0.0,0.0,4.0,1.0,1.0,0.0
26,"Hofstede, B",0.0,1.0,0.0,1.0,0.0,0.0
0,"Appleyard, R",1.0,2.0,19.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...
31,"Jacobson, P",0.0,0.0,0.0,0.0,1.0,1.0
32,"Janne, H",0.0,0.0,0.0,0.0,0.0,1.0
33,"Kant, E",0.0,0.0,0.0,0.0,0.0,1.0
35,"Kulischer, E",0.0,0.0,0.0,0.0,0.0,1.0


In [110]:
org_au_df[org_au_df['Sage Publishing'] > 0]


,author_surname_initial,Sage Publishing,Staatsdrukkerij,Wiley,Dutch Government,ICEM,REMP
0,"Appleyard, R",1.0,2.0,19.0,0.0,0.0,1.0
5,"Beyer, G",3.0,12.0,9.0,0.0,0.0,1.0
60,"Thomas, B",2.0,0.0,2.0,0.0,0.0,1.0
61,"Thomas, J",1.0,0.0,1.0,0.0,1.0,0.0
62,"Thompson, L",1.0,0.0,0.0,0.0,1.0,0.0


In [111]:
org_au_df[org_au_df['Wiley'] > 0]


,author_surname_initial,Sage Publishing,Staatsdrukkerij,Wiley,Dutch Government,ICEM,REMP
0,"Appleyard, R",1.0,2.0,19.0,0.0,0.0,1.0
1,"Avila, F",0.0,2.0,1.0,0.0,0.0,1.0
3,"Backer, J",0.0,0.0,1.0,0.0,0.0,1.0
4,"Besterman, W",0.0,0.0,2.0,0.0,1.0,0.0
5,"Beyer, G",3.0,12.0,9.0,0.0,0.0,1.0
6,"Borrie, W",0.0,1.0,1.0,0.0,0.0,1.0
9,"Carlin, J",0.0,0.0,2.0,0.0,1.0,0.0
10,Charry-Sam...,0.0,0.0,1.0,0.0,1.0,0.0
24,"Haveman, B",0.0,0.0,4.0,1.0,1.0,0.0
41,"Maselli, G",0.0,1.0,2.0,0.0,1.0,0.0


In [112]:
org_au_df[org_au_df['Staatsdrukkerij'] > 0]

,author_surname_initial,Sage Publishing,Staatsdrukkerij,Wiley,Dutch Government,ICEM,REMP
0,"Appleyard, R",1.0,2.0,19.0,0.0,0.0,1.0
1,"Avila, F",0.0,2.0,1.0,0.0,0.0,1.0
5,"Beyer, G",3.0,12.0,9.0,0.0,0.0,1.0
6,"Borrie, W",0.0,1.0,1.0,0.0,0.0,1.0
8,"Brink, Van...",0.0,2.0,0.0,0.0,0.0,1.0
13,"Edding, F",0.0,4.0,0.0,0.0,0.0,1.0
17,"Gadolin, D...",0.0,2.0,0.0,0.0,0.0,1.0
20,"Gottmann, J",0.0,1.0,0.0,0.0,0.0,1.0
22,"Groenman, S",0.0,2.0,0.0,0.0,0.0,1.0
26,"Hofstede, B",0.0,1.0,0.0,1.0,0.0,0.0


In [113]:
org_au_df.to_csv('../data/authors-organisations-overlap.csv')

In [114]:
au_org_df.to_csv('../data/authors-organisations-overlap-all.csv')

### My Section

In [115]:
overlap_df_total = org_au_df

overlap_df_total.rename(columns={'Sage Publishing':'IMR-Sage', 'Staatsdrukkerij':'REMP-Staatsdrukkerij', 'Wiley':'IM-Wiley',
       'Dutch Government':'Dutch_Government'}, inplace=True)
overlap_df_total.columns

article_cols = ['IMR-Sage', 'REMP-Staatsdrukkerij','IM-Wiley']
governance_cols = ['Dutch_Government', 'ICEM', 'REMP']
countcols = article_cols+governance_cols

In [116]:
overlap_df_total["network_count"] = overlap_df_total[countcols].replace(0,pd.NA).count(axis=1)

In [117]:
overlap_df_total["network_mean"] = overlap_df_total[countcols].mean(axis=1)


In [118]:
overlap_df_total["network_sum"] = overlap_df_total[countcols].sum(axis=1)

In [119]:
len(overlap_df_total)

72

In how many networks are people:

In [120]:
counted_values = overlap_df_total["network_count"].value_counts()
counted_values.sort_index(ascending=False)
counted_values

1    40
2    22
3     8
4     2
Name: network_count, dtype: int64

In [121]:
vals = list(counted_values.index)
vals.sort(reverse=True)
vals

[4, 3, 2, 1]

This means that most people are in one network and only 2 are in 4 networks. The total number of people is low (72), as we are (here) only interested in people who are in both the adminstrative cicuit and the academic cicuit of the discourse coalition. We have less data about people in the administrative cicuit. 

In [122]:
mean_values = overlap_df_total.network_mean.value_counts()
means = list(mean_values.index)
means.sort(reverse=True)


In [123]:
mean_values

0.166667    40
0.500000    12
0.333333    11
0.666667     3
0.833333     3
3.833333     1
4.166667     1
1.000000     1
Name: network_mean, dtype: int64

In [124]:
means

[4.166666666666667,
 3.8333333333333335,
 1.0,
 0.8333333333333334,
 0.6666666666666666,
 0.5,
 0.3333333333333333,
 0.16666666666666666]

In [125]:
board_df[['author_surname_initial', 'prs_category']]

,author_surname_initial,prs_category
0,"Beyer, G",academic
1,"Groenman, S",academic
2,"Zeegers, G",academic
3,"Hofstee, E",academic
4,"Bouman, P",academic
...,...,...
69,"Maselli, G",
70,Charry-Sam...,
71,"Escaler, N",
72,"Ndioro, N",


In [129]:
overlap_with_categories_df = pd.merge(left=overlap_df_total, right=board_df[['author_surname_initial', 'prs_category']],
         left_on=overlap_df_total.author_surname_initial, right_on='author_surname_initial', how='left')[
    ['author_surname_initial','IMR-Sage',
       'REMP-Staatsdrukkerij', 'IM-Wiley', 'Dutch_Government', 'ICEM', 'REMP',
       'network_sum', 'network_count', 'network_mean', 
       'prs_category']]
overlap_with_categories_df.drop_duplicates(inplace=True)

In [130]:
overlap_with_categories_df

,author_surname_initial,IMR-Sage,REMP-Staatsdrukkerij,IM-Wiley,Dutch_Government,ICEM,REMP,network_sum,network_count,network_mean,prs_category
0,"Appleyard, R",1.0,2.0,19.0,0.0,0.0,1.0,23.0,4,3.833333,academic
1,"Avila, F",0.0,2.0,1.0,0.0,0.0,1.0,4.0,3,0.666667,academic
2,"Baade, F",0.0,0.0,0.0,0.0,0.0,1.0,1.0,1,0.166667,academic
3,"Backer, J",0.0,0.0,1.0,0.0,0.0,1.0,2.0,2,0.333333,academic
4,"Besterman, W",0.0,0.0,2.0,0.0,1.0,0.0,3.0,2,0.500000,
...,...,...,...,...,...,...,...,...,...,...,...
69,"Vito, F",0.0,0.0,0.0,0.0,0.0,1.0,1.0,1,0.166667,academic
70,"Vitorino, A",0.0,0.0,0.0,0.0,1.0,0.0,1.0,1,0.166667,
71,"Weinberg, A",0.0,2.0,1.0,0.0,0.0,1.0,4.0,3,0.666667,academic
72,"Winkler, W",0.0,0.0,0.0,0.0,0.0,1.0,1.0,1,0.166667,academic


In [131]:
overlap_df_total.loc[overlap_df_total.network_count==4]

,author_surname_initial,IMR-Sage,REMP-Staatsdrukkerij,IM-Wiley,Dutch_Government,ICEM,REMP,network_count,network_mean,network_sum
0,"Appleyard, R",1.0,2.0,19.0,0.0,0.0,1.0,4,3.833333,23.0
5,"Beyer, G",3.0,12.0,9.0,0.0,0.0,1.0,4,4.166667,25.0


In [134]:
from IPython.core.display import display, HTML
style = "style='display:inline'"


for i in vals:
    if i>1: # we only want those who appear frequently
        res = overlap_df_total.loc[overlap_df_total.network_count==i]
        #res = res[numcols].apply(lambda x: x.round(0))
        display(res.style
                .set_table_attributes(style)
                .set_caption(f'<h1>{i} networks<h1>')
                .set_precision(0)
               .apply(lambda x:highlight_max(x, color='orange'), subset=article_cols, axis=0)
               .apply(lambda x:highlight_max(x, color='red'), subset=governance_cols, axis=0))

ValueError: Function <function <lambda> at 0x7fda866cbb70> returned the wrong shape.
Result has shape: (3,)
Expected shape:   (2, 3)

,author_surname_initial,IMR-Sage,REMP-Staatsdrukkerij,IM-Wiley,Dutch_Government,ICEM,REMP,network_count,network_mean,network_sum
1,"Avila, F",0,2,1,0,0,1,3,1,4
6,"Borrie, W",0,1,1,0,0,1,3,0,3
24,"Haveman, B",0,0,4,1,1,0,3,1,6
41,"Maselli, G",0,1,2,0,1,0,3,1,4
57,"Sauvy, A",0,1,1,0,0,1,3,0,3
60,"Thomas, B",2,0,2,0,0,1,3,1,5
61,"Thomas, J",1,0,1,0,1,0,3,0,3
69,"Weinberg, A",0,2,1,0,0,1,3,1,4


,author_surname_initial,IMR-Sage,REMP-Staatsdrukkerij,IM-Wiley,Dutch_Government,ICEM,REMP,network_count,network_mean,network_sum
3,"Backer, J",0,0,1,0,0,1,2,0,2
4,"Besterman, W",0,0,2,0,1,0,2,0,3
8,"Brink, Van Den, T",0,2,0,0,0,1,2,0,3
9,"Carlin, J",0,0,2,0,1,0,2,0,3
10,"Charry-Samper, H",0,0,1,0,1,0,2,0,2
13,"Edding, F",0,4,0,0,0,1,2,1,5
17,"Gadolin, De, A",0,2,0,0,0,1,2,0,3
20,"Gottmann, J",0,1,0,0,0,1,2,0,2
22,"Groenman, S",0,2,0,0,0,1,2,0,3
26,"Hofstede, B",0,1,0,1,0,0,2,0,2


In [137]:
# map unique values to color map from https://stackoverflow.com/questions/38249454/extract-rgb-or-6-digit-code-from-seaborn-palette
# and https://seaborn.pydata.org/tutorial/color_palettes.html
# and https://stackoverflow.com/questions/26139423/plot-different-color-for-different-categorical-levels-using-matplotlib
import seaborn as sns
# Unique category labels
color_labels = overlap_with_categories_df['prs_category'].unique()

# List of RGB triplets
rgb_values = sns.color_palette(palette="tab10", n_colors=len(color_labels))

# Map label to hex colors
color_map = dict(zip(color_labels, rgb_values.as_hex()))
pd.DataFrame().from_dict(color_map, orient='index').style.apply(lambda x: ['background-color: {}'.format(x[0])], axis=1)

,0
academic,#1f77b4
,#ff7f0e
unknown,#2ca02c
diplomat,#d62728


In [138]:
governance_cols

['Dutch_Government', 'ICEM', 'REMP']

In [239]:
from IPython.core.display import display, HTML
style = "style='display:inline'"

categories = pd.Categorical(overlap_with_categories_df.prs_category.unique())

res = overlap_with_categories_df.loc[overlap_with_categories_df.network_count>1]
res = res.sort_values(['network_count', 'network_mean', 'network_sum'], ascending=False)
#res = res[numcols].apply(lambda x: x.round(0))
display(res.style
        .set_table_attributes(style)
        .set_caption(f'<h1>network overlap<h1>')
        .set_precision(0)
        .hide_index()
        .apply(lambda x:highlight_max(x, color='orange'), subset=article_cols, axis=0)
        .apply(lambda x:highlight_max(x, color='red'), subset=governance_cols, axis=0)
        #.apply(lambda x:highlight_person(x, color='green',column='author_surname_initial', cond))
        .hide_columns(['prs_category']))

author_surname_initial,IMR-Sage,REMP-Staatsdrukkerij,IM-Wiley,Dutch_Government,ICEM,REMP,network_sum,network_count,network_mean
"Beyer, G",3,12,9,0,0,1,25,4,4
"Appleyard, R",1,2,19,0,0,1,23,4,4
"Haveman, B",0,0,4,1,1,0,6,3,1
"Thomas, B",2,0,2,0,0,1,5,3,1
"Avila, F",0,2,1,0,0,1,4,3,1
"Maselli, G",0,1,2,0,1,0,4,3,1
"Weinberg, A",0,2,1,0,0,1,4,3,1
"Borrie, W",0,1,1,0,0,1,3,3,0
"Sauvy, A",0,1,1,0,0,1,3,3,0
"Thomas, J",1,0,1,0,1,0,3,3,0


In [279]:
categories = color_labels
    
styled_res = res.style\
                .set_table_attributes(style)\
                .set_caption(f'<h1>network overlap<h1>')\
                .set_precision(0)\
                .hide_index()\
                .apply(lambda x:highlight_max(x, color='orange'), subset=article_cols, axis=0)\
                .apply(lambda x:highlight_max(x, color='red'), subset=governance_cols, axis=0)\
                #.apply(lambda x: ['background-color: {}'.format(color_map[x[10]])], axis=1, 
                #        subset=['author_surname_initial'])\
                #.hide_columns(['prs_category'])

tblhtml = styled_res.render()
        
# dphtml = r'<link rel="stylesheet" type="text/css" media="screen" href="css-table.css" />' + '\n'
# dphtml += dp.to_html()

with open('/Users/rikhoekstra/Downloads/datatable.html','w') as f:
     f.write(tblhtml)

## highlighting functions

(not all used, to be cleaned up)

In [133]:

# all this is stolen from https://pandas.pydata.org/pandas-docs/stable/user_guide/style.html 
# and a lot more
df = pd.DataFrame([[2,3,-3], [3,2,7], [2,4,4]], columns=list("ABC"))
def highlight_cel(val):
    """
    Takes a scalar and returns a string with
    the css property `'color: red'` for negative
    strings, black otherwise.
    """
    if val > 1:
        color = 'red'
    else:
        color = ''
    return 'background-color: %s' % color

def highlight_c(val, color='green'):
    color = color
    attr = 'background-color: %s' % color
    return [attr if v else '' for v in val]

def select_col(x):
    c1 = 'background-color: red'
    c2 = '' 
    #compare columns
    mask = x['prs_category' == 'academic']
    #DataFrame with same index and columns names as original filled empty strings
    df1 =  pd.DataFrame(c2, index=x.index, columns=x.columns)
    #modify values of df1 column by boolean mask
    df1.loc[mask, 'author_surname_initial'] = c1
    return df1
    


def highlight_max(data, color='yellow'):
    '''
    highlight the maximum in a Series or DataFrame
    '''
    attr = 'background-color: {}'.format(color)
    if data.ndim == 1:  # Series from .apply(axis=0) or axis=1
        if data.max() > 0:
            is_max = data == data.max()
            return [attr if v else '' for v in is_max]
    else:  # from .apply(axis=None)
        if data.max().max() > 0:
            is_max = data == data.max().max()
            return pd.DataFrame(np.where(is_max, attr, ''),
                                index=data.index, columns=data.columns)


def highlight_person(data, tcolumn, column, condition, color='red'):
    # works on rows (axis=1)
    attr = 'background-color: {}'.format(color)
    if data[column] == condition:
        return attr
        
#df.style.applymap(lambda x: color_negative_red(x))
# df.style.apply(lambda x: ["background-color: #ff33aa" 
#                           if (i >= 2 and (v > x.iloc[0] + x.iloc[1] 
#                                           or v < x.iloc[0] - x.iloc[1])) 
#                           else "" for i, v in enumerate(x)], axis = 1)

## Network per decade

The code above is all useful, but we need to add a temporal dimension to the networks. Therefore we subdivided the administrative networks ('boards') to decades in which we gave scored the people if they were in a network at a certain decade

In [38]:
decade_au_org_df = pd.merge(left=decade_au_pub_df, right=boarders, on='author_surname_initial', how='left').fillna(0)
decade_au_org_df.drop_duplicates(inplace=True)
decade_au_org_df.rename(columns={c:f"{c[0]}_{c[1]}" for c in decade_au_org_df.columns if type(c[1])==int }, inplace=True)

/Users/rikhoekstra/anaconda/envs/36/lib/python3.7/site-packages/pandas/core/reshape/merge.py:643: UserWarning: merging between different levels can give an unintended result (2 levels on the left,1 on the right)
  warnings.warn(msg, UserWarning)


In [39]:
colranges = {}
for i in range(1950,2010,10):
    colranges[f"{i}"] = [c for c in decade_au_org_df.columns if f"{i}" in c]

In [40]:
colranges

{'1950': [], '1960': [], '1970': [], '1980': [], '1990': [], '2000': []}

In [41]:
colorder=[]
for c in colranges.values():
    colorder.extend(c)
colorder

[]

In [42]:
decade_au_org_df.loc[decade_au_org_df.author_surname_initial.str.contains('Haveman')]

publisher        author_surname_initial Sage Publishing            \
issue_pub_decade                                   1960 1970 1980   
920                          Haveman, B             0.0  0.0  0.0   

publisher        Staatsdrukkerij                Wiley            
issue_pub_decade            1950 1960 1970 1980  1960 1970 1980  
920                          0.0  0.0  0.0  0.0   4.0  0.0  0.0

In [45]:
decade_au_org_df.columns

MultiIndex([('author_surname_initial',   ''),
            (       'Sage Publishing', 1960),
            (       'Sage Publishing', 1970),
            (       'Sage Publishing', 1980),
            (       'Staatsdrukkerij', 1950),
            (       'Staatsdrukkerij', 1960),
            (       'Staatsdrukkerij', 1970),
            (       'Staatsdrukkerij', 1980),
            (                 'Wiley', 1960),
            (                 'Wiley', 1970),
            (                 'Wiley', 1980)],
           names=['publisher', 'issue_pub_decade'])

In [65]:
cols = ['author_surname_initial']
cols.extend(colorder)

In [243]:
cols

['author_surname_initial',
 'author_surname_initial',
 'Sage Publishing_1960',
 'Sage Publishing_1970',
 'Sage Publishing_1980',
 'Staatsdrukkerij_1950',
 'Staatsdrukkerij_1960',
 'Staatsdrukkerij_1970',
 'Staatsdrukkerij_1980',
 'Wiley_1960',
 'Wiley_1970',
 'Wiley_1980',
 'REMP_1950',
 'REMP_1960',
 'REMP_1970',
 'REMP_1980',
 'REMP_1990',
 'REMP_2000',
 'Dutch Government_1950',
 'Dutch Government_1960',
 'Dutch Government_1970',
 'Dutch Government_1980',
 'Dutch Government_1990',
 'Dutch Government_2000',
 'ICEM_1950',
 'ICEM_1960',
 'ICEM_1970',
 'ICEM_1980',
 'ICEM_1990',
 'ICEM_2000']

In [70]:
overlap_df = decade_au_org_df[cols]

NotImplementedError: Index._join_level on non-unique index is not implemented

and then the count

In [51]:
overlap_df_total

NameError: name 'overlap_df_total' is not defined

In [52]:
overlap_df.sort_values(['count','mean','total'], ascending=False)

NameError: name 'overlap_df' is not defined

the value count of networks. For this we need the totalled dataframe, as people may occur in more decades. Which is useful in other respects

In [67]:
counted_values = overlap_df["count"].value_counts()
counted_values.sort_index(ascending=False)
vals = list(counted_values.index)
vals.sort(reverse=True)
vals

NameError: name 'overlap_df' is not defined

In [54]:
counted_values

NameError: name 'counted_values' is not defined

In [55]:
vals

NameError: name 'vals' is not defined

# Visualisation of overlap

In [56]:
pd.get_option('display.max_colwidth')

50

In [57]:
pd.set_option("display.max_colwidth",20)
cols1950 = [c for c in decade_au_org_df.columns if '1950' in c]
decade_au_org_df[cols1950].sum(axis=1)

decade_au_org_df.loc[decade_au_org_df[cols1950].sum(axis=1) > 0]

Empty DataFrame
Columns: [(author_surname_initial, ), (Sage Publishing, 1960), (Sage Publishing, 1970), (Sage Publishing, 1980), (Staatsdrukkerij, 1950), (Staatsdrukkerij, 1960), (Staatsdrukkerij, 1970), (Staatsdrukkerij, 1980), (Wiley, 1960), (Wiley, 1970), (Wiley, 1980)]
Index: []

In [58]:
colnames = decade_au_org_df.columns
colnames = ['author_surname_initial', 'Sage Publishing_1960',
       'Sage Publishing_1970', 'Sage Publishing_1980', 'Staatsdrukkerij_1950',
       'Staatsdrukkerij_1960', 'Staatsdrukkerij_1970', 'Staatsdrukkerij_1980',
       'Wiley_1960', 'Wiley_1970', 'Wiley_1980', 'REMP_1950', 'REMP_1960',
       'REMP_1970', 'REMP_1980', 'REMP_1990', 'REMP_2000',
       'Dutch Government_1950', 'Dutch Government_1960',
       'Dutch Government_1970', 'Dutch Government_1980',
       'Dutch Government_1990', 'Dutch Government_2000', 'ICEM_1950',
       'ICEM_1960', 'ICEM_1970', 'ICEM_1980', 'ICEM_1990', 'ICEM_2000']

In [59]:
colorder = {v:k for k,v in enumerate(colnames)}
colorder

{'author_surname_initial': 0,
 'Sage Publishing_1960': 1,
 'Sage Publishing_1970': 2,
 'Sage Publishing_1980': 3,
 'Staatsdrukkerij_1950': 4,
 'Staatsdrukkerij_1960': 5,
 'Staatsdrukkerij_1970': 6,
 'Staatsdrukkerij_1980': 7,
 'Wiley_1960': 8,
 'Wiley_1970': 9,
 'Wiley_1980': 10,
 'REMP_1950': 11,
 'REMP_1960': 12,
 'REMP_1970': 13,
 'REMP_1980': 14,
 'REMP_1990': 15,
 'REMP_2000': 16,
 'Dutch Government_1950': 17,
 'Dutch Government_1960': 18,
 'Dutch Government_1970': 19,
 'Dutch Government_1980': 20,
 'Dutch Government_1990': 21,
 'Dutch Government_2000': 22,
 'ICEM_1950': 23,
 'ICEM_1960': 24,
 'ICEM_1970': 25,
 'ICEM_1980': 26,
 'ICEM_1990': 27,
 'ICEM_2000': 28}

In [60]:
import re
namemap = {"Sage_": "IMR",
           "Sage Publishing_": "IMR",
           "Wiley_":"IM",
           "Staatsdrukkerij_": "B",
           "REMP_": "R",
           "Dutch Government_": "NL",
           "ICEM_":"I"}

def sub_col(x):
    z = x
    for item in namemap:
        if item in x:
            z = x.replace(item, '')
            z = z + namemap[item]
    return z

ncol = {item:sub_col(item)for item in colnames}
ncol

{'author_surname_initial': 'author_surname_initial',
 'Sage Publishing_1960': '1960IMR',
 'Sage Publishing_1970': '1970IMR',
 'Sage Publishing_1980': '1980IMR',
 'Staatsdrukkerij_1950': '1950B',
 'Staatsdrukkerij_1960': '1960B',
 'Staatsdrukkerij_1970': '1970B',
 'Staatsdrukkerij_1980': '1980B',
 'Wiley_1960': '1960IM',
 'Wiley_1970': '1970IM',
 'Wiley_1980': '1980IM',
 'REMP_1950': '1950R',
 'REMP_1960': '1960R',
 'REMP_1970': '1970R',
 'REMP_1980': '1980R',
 'REMP_1990': '1990R',
 'REMP_2000': '2000R',
 'Dutch Government_1950': '1950NL',
 'Dutch Government_1960': '1960NL',
 'Dutch Government_1970': '1970NL',
 'Dutch Government_1980': '1980NL',
 'Dutch Government_1990': '1990NL',
 'Dutch Government_2000': '2000NL',
 'ICEM_1950': '1950I',
 'ICEM_1960': '1960I',
 'ICEM_1970': '1970I',
 'ICEM_1980': '1980I',
 'ICEM_1990': '1990I',
 'ICEM_2000': '2000I'}

In [61]:
org = ["REMP_","Dutch Government_", "ICEM_"]
pub = ["Sage_","Sage Publishing_","Wiley_","Staatsdrukkerij_"]
decades = [1950, 1960, 1970, 1980, 1990]


In [62]:
pd.set_option("display.max_colwidth",14)
au_cols1950 = ['author_surname_initial'] + cols1950

In [166]:
d_cols = {}
for d in decades:
    d_cols[f"{d}"] = [c for c in colnames if f"{d}" in c and c in decade_au_org_df.columns]
d_cols

{'1950': ['Staatsdrukkerij_1950',
  'REMP_1950',
  'Dutch Government_1950',
  'ICEM_1950'],
 '1960': ['Sage Publishing_1960',
  'Staatsdrukkerij_1960',
  'Wiley_1960',
  'REMP_1960',
  'Dutch Government_1960',
  'ICEM_1960'],
 '1970': ['Sage Publishing_1970',
  'Staatsdrukkerij_1970',
  'Wiley_1970',
  'REMP_1970',
  'Dutch Government_1970',
  'ICEM_1970'],
 '1980': ['Sage Publishing_1980',
  'Staatsdrukkerij_1980',
  'Wiley_1980',
  'REMP_1980',
  'Dutch Government_1980',
  'ICEM_1980'],
 '1990': ['REMP_1990', 'ICEM_1990'],
 '2000': ['REMP_2000', 'ICEM_2000']}

In [174]:
decade_au_org_df[selcols].sum(axis=1) > 0

0       False
1       False
2       False
3       False
4       False
        ...  
2494    False
2495    False
2496     True
2497    False
2498    False
Length: 2498, dtype: bool

In [201]:
len(tempdf.loc[tempdf.sum])

2498

In [223]:
sums = sel.sum()
l = list(sums.loc[sums.values==0].index)
for c in sel.columns:
    if c != 'author_surname_initial':
        if c in l:
            sel.drop(c, axis=1, inplace=True)
sel

,author_surname_initial


In [281]:
#fifties = decade_au_org_df.loc[decade_au_org_df[cols1950].sum(axis=1) > 0][au_cols1950]
#fifties.sort_values(cols1950, inplace=True, ascending=False)
#fifties.replace()
import warnings
warnings.filterwarnings('ignore')
for key in d_cols:
    selcols = d_cols[key]
    tempdf = decade_au_org_df[['author_surname_initial'] + selcols]
    tempdf['total_networks'] = decade_au_org_df[selcols].replace(0, pd.NA).count(axis=1)
    sel = tempdf.loc[tempdf.total_networks>1].sort_values("total_networks", ascending=False)
    sums = sel.sum()
    l = list(sums.loc[sums.values==0].index)
    for c in sel.columns:
        if c != 'author_surname_initial':
            if c in l:
                try:
                    sel.drop(c, axis=1, inplace=True)
                except KeyError:
                    pass
                try:
                    selcols.remove(c)
                except ValueError:
                    pass
    styled_res = sel.style\
                    .set_table_attributes(style)\
                    .set_precision(0)\
                    .apply(lambda x:highlight_c(x, color=color_map[key]), subset=selcols, axis=1)
    display(HTML(f"<H1>Network overlap {key}</H1>"))
    display(styled_res
        )
    tblhtml = styled_res.render()
        
# dphtml = r'<link rel="stylesheet" type="text/css" media="screen" href="css-table.css" />' + '\n'
# dphtml += dp.to_html()

    with open(f'/Users/rikhoekstra/Downloads/datatable_{key}.html','w') as f:
         f.write(tblhtml)
           

,author_surname_initial,Staatsdrukkerij_1950,REMP_1950,total_networks
207,"Beyer, G",10,1,2
277,"Brink, Van Den, T",2,1,2
719,"Gadolin, De, A",2,1,2
817,"Gottmann, J",1,1,2
848,"Groenman, S",2,1,2
978,"Hofstee, E",4,1,2
1026,"Hyrenius, H",1,1,2
1041,"Isaac, J",2,1,2
1410,"Mackenroth, G",1,1,2
1667,"Nixon, J",1,1,2


,author_surname_initial,Sage Publishing_1960,Staatsdrukkerij_1960,Wiley_1960,REMP_1960,ICEM_1960,total_networks
207,"Beyer, G",1,2,5,1,0,4
2377,"Weinberg, A",0,1,1,1,0,3
1878,"Richardson, A",1,2,2,0,0,3
82,"Appleyard, R",0,0,1,1,0,2
1769,"Perotti, A",4,0,1,0,0,2
2227,"Timlin, M",0,1,1,0,0,2
2216,"Thomas, J",0,0,1,0,1,2
2215,"Thomas, B",0,0,2,1,0,2
2193,"Taft, R",1,1,0,0,0,2
1998,"Sauvy, A",0,0,1,1,0,2


,author_surname_initial,Sage Publishing_1970,Staatsdrukkerij_1970,Wiley_1970,REMP_1970,ICEM_1970,total_networks
207,"Beyer, G",1,0,3,1,0,3
190,"Bernard, W",4,1,5,0,0,3
1460,"Maselli, G",0,1,1,0,1,3
50,"Alexander, R",1,1,0,0,0,2
1574,"Moncarz, R",1,0,6,0,0,2
2375,"Weiermair, K",1,0,2,0,0,2
2243,"Toren, N",1,0,1,0,0,2
2193,"Taft, R",1,0,1,0,0,2
2134,"Stark, T",4,0,1,0,0,2
1906,"Rogers, T",2,0,3,0,0,2


,author_surname_initial,Sage Publishing_1980,Staatsdrukkerij_1980,Wiley_1980,REMP_1980,ICEM_1980,total_networks
207,"Beyer, G",1,0,1,1,0,3
24,"Adepoju, A",1,0,1,0,0,2
1849,"Rath, J",6,0,1,0,0,2
1480,"Mccready, D",2,0,1,0,0,2
1546,"Miller, M",7,0,2,0,0,2
1554,"Mirdal, G",1,0,1,0,0,2
1574,"Moncarz, R",0,1,1,0,0,2
1728,"Papademetriou, D",4,0,5,0,0,2
1736,"Passaris, C",1,0,3,0,0,2
1758,"Penninx, R",2,0,2,0,0,2


,author_surname_initial


,author_surname_initial


In [286]:
# map unique values to color map from https://stackoverflow.com/questions/38249454/extract-rgb-or-6-digit-code-from-seaborn-palette
# and https://seaborn.pydata.org/tutorial/color_palettes.html
# and https://stackoverflow.com/questions/26139423/plot-different-color-for-different-categorical-levels-using-matplotlib
import seaborn as sns
# Unique category labels
color_labels = list(d_cols.keys())

# List of RGB triplets
rgb_values = sns.color_palette(palette="tab10", n_colors=len(color_labels))

# Map label to hex colors
color_map = dict(zip(color_labels, rgb_values.as_hex()))
pd.DataFrame().from_dict(color_map, orient='index').style.apply(lambda x: ['background-color: {}'.format(x[0])], axis=1)

,0
1950,#1f77b4
1960,#ff7f0e
1970,#2ca02c
1980,#d62728
1990,#9467bd
2000,#8c564b


In [249]:
networkers = {x:res.loc[res.network_count==x] for x in res.network_count.unique()}

In [254]:
list(networkers[4].author_surname_initial)

['Beyer, G', 'Appleyard, R']

In [261]:
decade_au_org_df\
    .loc[decade_au_org_df.author_surname_initial.isin(list(networkers[4].author_surname_initial))]\
    .rename(columns=ncol)

,author_surname_initial,1960IMR,1970IMR,1980IMR,1950B,1960B,1970B,1980B,1960IM,1970IM,1980IM,1950R,1960R,1970R,1980R,1960I,1970I,1980I,1990I,2000I
82,"Appleyard, R",0.0,0.0,1.0,2.0,0.0,0.0,0.0,1.0,1.0,3.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
207,"Beyer, G",1.0,1.0,1.0,10.0,2.0,0.0,0.0,5.0,3.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0


In [267]:
z = n_res.apply(lambda x:highlight_c(x, color=color_map[key]), axis=1)

In [ ]:
z[]

In [303]:

#fifties = decade_au_org_df.loc[decade_au_org_df[cols1950].sum(axis=1) > 0][au_cols1950]
#fifties.sort_values(cols1950, inplace=True, ascending=False)
#fifties.replace()
import warnings
warnings.filterwarnings('ignore')
sums = decade_au_org_df.sum()
l = list(sums.loc[sums.values==0].index)
for c in decade_au_org_df.columns:
    if c != 'author_surname_initial':
        if c in l:
            try:
                decade_au_org_df.drop(c, axis=1, inplace=True)
            except KeyError:
                pass
for networker in networkers:
    n_res = decade_au_org_df\
        .loc[decade_au_org_df.author_surname_initial.isin(list(networkers[networker].author_surname_initial))]\
        #.rename(columns=ncol)
    styled_res = n_res.style\
                    .set_table_attributes(style)\
                    .set_precision(0)
    for key in d_cols:
        selcols = d_cols[key]
        #tempdf = decade_au_org_df[['author_surname_initial'] + selcols]
        #tempdf['total_networks'] = decade_au_org_df[selcols].replace(0, pd.NA).count(axis=1)
        #sel = tempdf.loc[tempdf.total_networks>1].sort_values("total_networks", ascending=False)
        styled_res.apply(lambda x:highlight_c(x, color=color_map[str(1950)]), subset=[c for c in n_res.columns if '1950' in c
                                                                                 ], axis=1)\
        .apply(lambda x:highlight_c(x, color=color_map[str(1960)]), subset=[c for c in n_res.columns if '1960' in c], axis=1)\
        .apply(lambda x:highlight_c(x, color=color_map[str(1970)]), subset=[c for c in n_res.columns if '1970' in c], axis=1)\
        .apply(lambda x:highlight_c(x, color=color_map[str(1980)]), subset=[c for c in n_res.columns if '1980' in c], axis=1)\
        .apply(lambda x:highlight_c(x, color=color_map[str(1990)]), subset=[c for c in n_res.columns if '1990' in c], axis=1)\
        .apply(lambda x:highlight_c(x, color=color_map[str(2000)]), subset=[c for c in n_res.columns if '2000' in c], axis=1)
    display(HTML(f"<H1>Network overlap {networker} networks</H1>"))
    display(styled_res)
    tblhtml = styled_res.render()
        
# dphtml = r'<link rel="stylesheet" type="text/css" media="screen" href="css-table.css" />' + '\n'
# dphtml += dp.to_html()

    with open(f'/Users/rikhoekstra/Downloads/datatable_{networker}.html','w') as f:
          f.write(tblhtml)
           

,author_surname_initial,Sage Publishing_1960,Sage Publishing_1970,Sage Publishing_1980,Staatsdrukkerij_1950,Staatsdrukkerij_1960,Staatsdrukkerij_1970,Staatsdrukkerij_1980,Wiley_1960,Wiley_1970,Wiley_1980,REMP_1950,REMP_1960,REMP_1970,REMP_1980,ICEM_1960,ICEM_1970,ICEM_1980,ICEM_1990,ICEM_2000
82,"Appleyard, R",0,0,1,2,0,0,0,1,1,3,0,1,0,0,0,0,0,0,0
207,"Beyer, G",1,1,1,10,2,0,0,5,3,1,1,1,1,1,0,0,0,0,0


,author_surname_initial,Sage Publishing_1960,Sage Publishing_1970,Sage Publishing_1980,Staatsdrukkerij_1950,Staatsdrukkerij_1960,Staatsdrukkerij_1970,Staatsdrukkerij_1980,Wiley_1960,Wiley_1970,Wiley_1980,REMP_1950,REMP_1960,REMP_1970,REMP_1980,ICEM_1960,ICEM_1970,ICEM_1980,ICEM_1990,ICEM_2000
103,"Avila, F",0,0,0,2,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0
250,"Borrie, W",0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0
918,"Haveman, B",0,0,0,0,0,0,0,4,0,0,0,0,0,0,1,0,0,0,0
1460,"Maselli, G",0,0,0,0,0,1,0,1,1,0,0,0,0,0,1,1,1,0,0
1998,"Sauvy, A",0,0,0,1,0,0,0,1,0,0,1,1,0,0,0,0,0,0,0
2215,"Thomas, B",0,2,0,0,0,0,0,2,0,0,0,1,0,0,0,0,0,0,0
2216,"Thomas, J",0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0
2377,"Weinberg, A",0,0,0,1,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0


,author_surname_initial,Sage Publishing_1960,Sage Publishing_1970,Sage Publishing_1980,Staatsdrukkerij_1950,Staatsdrukkerij_1960,Staatsdrukkerij_1970,Staatsdrukkerij_1980,Wiley_1960,Wiley_1970,Wiley_1980,REMP_1950,REMP_1960,REMP_1970,REMP_1980,ICEM_1960,ICEM_1970,ICEM_1980,ICEM_1990,ICEM_2000
111,"Backer, J",0,0,0,0,0,0,0,1,0,0,1,1,0,0,0,0,0,0,0
203,"Besterman, W",0,0,0,0,0,0,0,2,0,0,0,0,0,0,1,1,0,0,0
277,"Brink, Van Den, T",0,0,0,2,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
329,"Carlin, J",0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,1,1,0,0
361,"Charry-Samper, H",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0
579,"Edding, F",0,0,0,4,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
719,"Gadolin, De, A",0,0,0,2,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
817,"Gottmann, J",0,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0
848,"Groenman, S",0,0,0,2,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0
977,"Hofstede, B",0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


## Old Analysis, Needs Redoing

### Looking at author overlap between journal

There following 212 authors who have published in both journals:

In [191]:
g = df_split[df_split.author_surname_initial.isna() == False].groupby(['author_surname_initial', 'journal']).journal.count()
df_overlap = g.unstack('journal').fillna(0.0)
df_overlap[(df_overlap.IM > 0) & (df_overlap.IMR > 0)]


NameError: name 'df_split' is not defined

There are 54 authors who published at least two articles in each journal. Here we see Gunther Beijer and and Jerzy Zubrzicki, who are also directly involved in the REMP publications:

In [192]:
g = df_split[df_split.author_surname_initial.isna() == False].groupby(['author_surname_initial', 'journal']).journal.count()
df_overlap = g.unstack('journal').fillna(0.0)
print('number of authors with at least two articles in both journals:', 
      len(df_overlap[(df_overlap.IM > 1) & (df_overlap.IMR > 1)]))

df_overlap[(df_overlap.IM > 1) & (df_overlap.IMR > 1)]

NameError: name 'df_split' is not defined

In [53]:
print('Number of authors in both journals combined:', len(df_overlap))
print('Number of authors in International Migration:', len(df_overlap[df_overlap.IM > 0]))
print('Number of authors in International Migration Review:', len(df_overlap[df_overlap.IMR > 0]))


NameError: name 'df_overlap' is not defined

The 212 authors with articles in both journals represent 18% of all IM authors ($212 / 1201 = 0.18$) and 8% ($212 / 2693 = 0.08$) of all IMR authors. 

In [54]:
print(round(212/1201, 2))
print(round(212/2693, 2))

0.18
0.08


In [55]:
print('Number of persons who published multiple articles in IM:', len(df_overlap[df_overlap.IM > 1]))

print('Number of multi-IM authors who also published in IMR:', len(df_overlap[(df_overlap.IM > 1) & (df_overlap.IMR > 0)]))

NameError: name 'df_overlap' is not defined

In [56]:
print('Number of persons who published multiple articles in IMR:', len(df_overlap[df_overlap.IMR > 1]))

print('Number of multi-IMR authors who also published in IM:', len(df_overlap[(df_overlap.IMR > 1) & (df_overlap.IM > 0)]))

NameError: name 'df_overlap' is not defined

In [57]:
print('IM:', len(df_overlap[df_overlap.IM > 1]) / len(df_overlap[df_overlap.IM > 0]))
print('IMR:', len(df_overlap[df_overlap.IMR > 1]) / len(df_overlap[df_overlap.IMR > 0]))

print('IM:', len(df_overlap[df_overlap.IM > 4]) / len(df_overlap[df_overlap.IM > 0]))
print('IMR:', len(df_overlap[df_overlap.IMR > 4]) / len(df_overlap[df_overlap.IMR > 0]))



NameError: name 'df_overlap' is not defined

In [58]:
print(df_overlap[df_overlap.IMR > 1].shape)
df_overlap[df_overlap.IMR > 1].IM.value_counts()

NameError: name 'df_overlap' is not defined

In [59]:
print(df_overlap[df_overlap.IM > 1].shape)
df_overlap[df_overlap.IM > 1].IMR.value_counts()

NameError: name 'df_overlap' is not defined

In [60]:
305/371
660/769

0.8582574772431729

In [61]:
df_overlap.plot(kind='scatter', x='IM', y='IMR')

NameError: name 'df_overlap' is not defined

In [62]:
g = df_split[(df_split.author_surname_initial.isna() == False) & (df_split.author_surname_initial != '')].groupby(['author_surname_initial', 'journal']).journal.count()
df_overlap = g.unstack('journal').fillna(0.0)
df_overlap

NameError: name 'df_split' is not defined

In [63]:
df_split[df_split.author_surname_initial.isna() == False]

NameError: name 'df_split' is not defined

# Junkyard

In [64]:
import seaborn as sns

cm = sns.light_palette("green", as_cmap=True)
s = oefendf.style.background_gradient(cmap=cm)
s

NameError: name 'oefendf' is not defined

In [ ]:
# map unique values to color map from https://stackoverflow.com/questions/38249454/extract-rgb-or-6-digit-code-from-seaborn-palette
# and https://seaborn.pydata.org/tutorial/color_palettes.html
# and https://stackoverflow.com/questions/26139423/plot-different-color-for-different-categorical-levels-using-matplotlib

# Unique category labels
color_labels = oefendf['prs_category'].unique()

# List of RGB triplets
rgb_values = sns.color_palette(palette="tab10", n_colors=len(color_labels))

# Map label to hex colors
color_map = dict(zip(color_labels, rgb_values.as_hex()))
pd.DataFrame().from_dict(color_map, orient='index').style.apply(lambda x: ['background-color: {}'.format(x[0])], axis=1)

In [ ]:
oefendf['cat'] = pd.Categorical(oefendf.prs_category)
oefendf.style.apply(
    lambda x: ['background-color: {}'.format(color_map[i]) for i in oefendf['cat']], 
    subset=['author_surname_initial']
)